## Task 2: Is Rectangle?

The objective of this task is to verify if 4 positive integers can represents the length of the sides of a rectangle, by using quantum computing.

Approach #1: create a quantum circuit to compare two bitstrings, and use it for each pair of the given numbers. In such an approach, I encode each bit in a different qubit, and implement a quantum circuit which compares the content of the qubits. Since each qubits is in a base state, measurement will give correct output with probability 1. 

### Approach #1


In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit import Aer, transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from qiskit.circuit.classicalfunction import classical_function
from qiskit.circuit.classicalfunction.types import Int1

In [2]:
"""
    Such a function returns a quantum circuit which compares the content of two qubits (in a base state).
    It does so by implement the logical boolean function:
    c = a AND b or NOT a AND NOT b
    by using qiskit.circuit.classical_function. It can also be designed by hand, by using CCX gates. 
"""

@classical_function
def qubit_comparator(a: Int1, b: Int1) -> Int1:
    return ((a and b) or (not a and not b))

In [3]:
print(qubit_comparator.synth())

               
q_0: ──■───────
       │       
q_1: ──┼────o──
     ┌─┴─┐┌─┴─┐
q_2: ┤ X ├┤ X ├
     └───┘└───┘


In [4]:
"""
    Such a function implements a quantum circuit which compares two registers of N qubits.
    @param N : number of qubits
"""
def register_comparator(N: int, value1: str, value2: str):
    assert N > 0
    assert len(value1) == N
    assert len(value1) == len(value2)
    
    circuit = QuantumCircuit(2*N+N, N)
    qubit_comp = qubit_comparator.synth()
    
    circuit.initialize(Statevector.from_label(value1), range(N))
    circuit.initialize(Statevector.from_label(value2), range(N,2*N))
    for i in range(N): 
        circuit.append(qubit_comp, [i, i+N, i + 2*N])
        circuit.measure(i + 2*N, i)
    return circuit

In [5]:
"""
    This function receives as input two positive integers and it verifies if they are equal, by using a quantum circuit.
    @params value1 : positive integer
    @params value2 : positive integer
    @returns boolean set to True if value1 equals value2, False otherwise.
"""
def check_numbers(value1: int, value2: int) -> bool:
    assert value1 > 0
    assert value2 > 0
    
    bw1 = np.ceil(np.log2(value1))+1
    bw2 = np.ceil(np.log2(value2))+1
    bitwidth = int(max(bw1, bw2))
    
    format_string = '{0:0' + str(bitwidth) + 'b}'
    str1 = format_string.format(value1)
    str2 = format_string.format(value2)
    #print(bitwidth,str1, str2)
    quantum_circuit = register_comparator(bitwidth, str1, str2)
    print("QUANTUM CIRCUIT TO CHECK %d and %d:" % (value1, value2))
    print(quantum_circuit)
    # use local simulator
    aer_sim = Aer.get_backend('aer_simulator')
    shots = 1 #we get correct output with probability 1
    quantum_circuit = transpile(quantum_circuit, backend=aer_sim)
    qobj = assemble(quantum_circuit, shots=shots)
    results = aer_sim.run(qobj).result()
    counts = results.get_counts()
    return "1" * bitwidth in counts
    

In [6]:
def is_rectangle (A: int, B: int, C: int, D: int):
    """
        A : integer value that is one side of the rectangle.
        B : integer value that is one side of the rectangle.
        C : integer value that is one side of the rectangle.
        D : integer value that is one side of the rectangle.
        Return if is a rectangle returns 1 else 0. 
    """
    assert A > 0 and B > 0 and C > 0 and D > 0
    
    AisB = check_numbers(A,B)
    if AisB and check_numbers(C,D):
        return True
    AisC = check_numbers(A,C)
    if AisC and check_numbers(B,D):
        return True
    AisD = check_numbers(A,D)
    if AisD and check_numbers(B,C):
        return True
    return False


In [13]:
result = is_rectangle(10,20,10,10)

QUANTUM CIRCUIT TO CHECK 10 and 20:
      »
 q_0: »
      »
 q_1: »
      »
 q_2: »
      »
 q_3: »
      »
 q_4: »
      »
 q_5: »
      »
 q_6: »
      »
 q_7: »
      »
 q_8: »
      »
 q_9: »
      »
q_10: »
      »
q_11: »
      »
q_12: »
      »
q_13: »
      »
q_14: »
      »
q_15: »
      »
q_16: »
      »
q_17: »
      »
 c: 6/»
      »
«      ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐»
« q_0: ┤0                                                                                                                                             ├»
«      │                                                                                                                                              │»
« q_1: ┤1                                                                                                                                             ├»
«      │                                

In [14]:
print("The output is:", result)

The output is: False
